In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE40911"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE40911"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE40911.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE40911.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE40911.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations (RCC malignancy)"
!Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microarrays. Oligoarrays with 44k-probes were used to interrogate 17 RCC samples (14 clear cell, 2 papillary, 1 chromophobe subtypes) split into four pools. Meta-analyses were performed by taking the genomic coordinates of the RCC-expressed lncRNAs, a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from microarray experiments
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait data (Kidney_Chromophobe)
# Looking at the sample characteristics dict, we can see "tissue: adjacent nontumor kidney tissue" vs "tissue: primary kidney tumor" at index 2
trait_row = 2  # Using the tissue field since we want to compare tumor vs non-tumor

# Define conversion function for trait
def convert_trait(value):
    if not isinstance(value, str):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Convert to binary: 1 for tumor (case), 0 for non-tumor (control)
    if 'tumor' in value.lower() and 'nontumor' not in value.lower() and 'non-tumor' not in value.lower():
        return 1
    elif 'nontumor' in value.lower() or 'non-tumor' in value.lower() or 'adjacent' in value.lower():
        return 0
    return None

# 2.2 Age data
age_row = 4  # Age at surgery is at index 4

def convert_age(value):
    if not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        # Extract numeric age value
        # The format seems to be "age at surgery (yrs): XX"
        return float(value)
    except ValueError:
        return None

# 2.3 Gender data
gender_row = 3  # Gender is at index 3

def convert_gender(value):
    if not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Convert to binary: 0 for female, 1 for male
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical_data from the sample characteristics dictionary shown in the output
    # The dictionary format is {row_index: [values_for_each_sample]}
    sample_chars_dict = {
        0: ['patient identifier: 3', 'patient identifier: 5', 'patient identifier: 8', 'patient identifier: 9', 'patient identifier: 10', 'patient identifier: 11', 'patient identifier: 24', 'patient identifier: 26', 'patient identifier: 28', 'patient identifier: 30', 'patient identifier: 31'],
        1: ['disease: clear cell renal cell carcinoma (RCC)'],
        2: ['tissue: adjacent nontumor kidney tissue', 'tissue: primary kidney tumor'],
        3: ['gender: female', 'gender: male'],
        4: ['age at surgery (yrs): 78', 'age at surgery (yrs): 53', 'age at surgery (yrs): 71', 'age at surgery (yrs): 39', 'age at surgery (yrs): 34', 'age at surgery (yrs): 51', 'age at surgery (yrs): 75', 'age at surgery (yrs): 40', 'age at surgery (yrs): 50'],
        5: ['patient status: cancer-specific death', 'patient status: dead from other causes', 'patient status: alive without cancer', 'patient status: alive with cancer', 'fuhrman grade: IV', 'fuhrman grade: III', 'fuhrman grade: II'],
        6: [np.nan, 'tumor size (cm): 6', 'tumor size (cm): 8', 'tumor size (cm): 5', 'tumor size (cm): 6.5', 'tumor size (cm): 7', 'tumor size (cm): 15', 'tumor size (cm): 8.5'],
        7: [np.nan, 'necrosis: yes', 'necrosis: no'],
        8: [np.nan, 'capsule infiltration: yes', 'capsule infiltration: no'],
        9: [np.nan, 'tnm classification (t): 3c', 'tnm classification (t): 2', 'tnm classification (t): 3a', 'tnm classification (t): 1b', 'tnm classification (t): 3b', 'tnm classification (t): 1'],
        10: [np.nan, 'tnm classification (n): no data available', 'tnm classification (n): 1', 'tnm classification (n): 0', 'tnm classification (n): 2'],
        11: [np.nan, 'tnm classification (m): 1', 'tnm classification (m): no data available'],
        12: [np.nan, 'organ metastasis at surgery: endocava, bones', 'organ metastasis at surgery: liver', 'organ metastasis at surgery: no data available', 'organ metastasis at surgery: lung, adjacent tissues'],
        13: [np.nan, 'organ metastasis after surgery: no data available', 'organ metastasis after surgery: liver, spleen', 'organ metastasis after surgery: brain, lung, bones'],
        14: [np.nan, 'patient status: cancer-specific death', 'patient status: dead from other causes', 'patient status: alive without cancer', 'patient status: alive with cancer']
    }
    
    # Create a properly structured DataFrame that can be used with geo_select_clinical_features
    clinical_data = pd.DataFrame()
    for i, values in sample_chars_dict.items():
        clinical_data[i] = pd.Series(values)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{0: [nan, 10.0, nan], 1: [nan, nan, nan], 2: [nan, nan, nan], 3: [nan, nan, nan], 4: [nan, 34.0, nan], 5: [nan, nan, nan], 6: [nan, 6.5, nan], 7: [nan, nan, nan], 8: [nan, nan, nan], 9: [nan, nan, nan], 10: [nan, 2.0, nan], 11: [nan, nan, nan], 12: [nan, nan, nan], 13: [nan, nan, nan], 14: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE40911.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 3055 rows
First 20 gene IDs:
Index(['3', '4', '6', '7', '9', '10', '15', '16', '17', '19', '20', '21', '22',
       '23', '26', '27', '31', '32', '33', '35'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers in the gene expression data
# The identifiers appear to be numeric values (e.g., '3', '4', '6', etc.)
# These are not standard human gene symbols, which typically look like BRCA1, TP53, etc.
# These numeric IDs likely need to be mapped to proper gene symbols

# This is likely to be Entrez Gene IDs or some other numeric identifier system
# Proper gene symbols are alphabetic (like BRCA1, TP53) not purely numeric

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 139072 rows

Gene annotation preview (first few rows):
{'ID': ['910', '4260', '1981', '2381', '4288'], 'GB_ACC': ['BE833259', 'BE702227', 'BF364095', 'BE081005', 'AW880607'], 'SPOT_TYPE': ['Exonic', 'Exonic', 'Exonic', 'Exonic', 'Exonic'], 'GENE_ID': [85439.0, 2776.0, 84131.0, 2776.0, 54768.0], 'GENE_SYMBOL': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN'], 'GENE_ANNOTATION': ['stonin 2', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'centrosomal protein 78kDa', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'hydrocephalus inducing homolog 2 (mouse); hydrocephalus inducing homolog (mouse)'], 'CPC_CODING_POTENTIAL': ['noncoding', 'noncoding', 'noncoding', 'noncoding', '-'], 'SEQUENCE': ['CTGATCCGCTTAAGCTTAGTATGTTTGAGTGTGTAATTTTAGTTTCTTTTCTGGTTGTATTTGTGGTAGTCAGATGTGTTGGATTGATTCCAACTGGACAGAGTAAGGAATTCCAGCATCCTCTTCCTGCTTGCTCGTGTTACCCCACAGATCAAACCCTCAATTCTAGTTGGGGATGCTGTCTAGCCCCACACCATGACTGAAGCCTTAAGCACTGTTGCG

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the mapping columns
# From the gene annotation preview, we need to map from 'ID' to 'GENE_SYMBOL'
# The gene expression data also uses 'ID' as its index
print("Mapping gene identifiers to gene symbols...")

# 2. Extract the mapping columns and create a mapping dataframe
prob_col = 'ID'  # Column with probe identifiers
gene_col = 'GENE_SYMBOL'  # Column with gene symbols

# Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created gene mapping with {len(mapping_df)} entries")

# Preview the mapping dataframe
print("\nGene mapping preview (first few rows):")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level data to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Successfully converted to gene expression data with {len(gene_data.index)} genes")

# Preview the gene expression data
print("\nGene expression data preview (first few genes):")
print(preview_df(gene_data))

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping gene identifiers to gene symbols...
Created gene mapping with 3169 entries

Gene mapping preview (first few rows):
{'ID': ['910', '4260', '1981', '2381', '4288'], 'Gene': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN']}

Converting probe-level measurements to gene expression data...


Successfully converted to gene expression data with 2502 genes

Gene expression data preview (first few genes):
{'GSM1004655': [11.611, 10.926954546000001, 5.866136364, 2.304568182, 3.128022727], 'GSM1004656': [12.39861364, 15.123068182, 6.7795, 2.531636364, 4.258579545], 'GSM1004657': [8.044136364, 9.524568182, 7.152886364, 4.169409091, 5.121579545], 'GSM1004658': [8.556590909, 9.290693181, 7.174977273, 3.884727273, 5.776840909], 'GSM1004659': [7.704806818, 16.374318182, 5.923522727, 0.992409091, 5.195102273], 'GSM1004660': [6.522909091, 9.687295455000001, 7.814090909, 2.001590909, 6.675045455], 'GSM1004661': [9.999795455, 10.968193182, 5.647227273, 2.02575, 5.629681818], 'GSM1004662': [9.7725, 12.251704546, 5.936204545, 4.709954545, 5.837704545], 'GSM1004663': [7.132090909, 9.254681818, 5.132613636, 2.187363636, 6.154090909], 'GSM1004664': [5.202318182, 9.108636364, 8.291772727, 2.650977273, 6.749909091], 'GSM1004665': [7.729409091, 11.017318182, 6.432477273, 2.658306818, 3.019965909

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Load clinical data from Step 2 (from the location where we saved it)
print("\nLoading clinical data...")
try:
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        print("Clinical data preview:")
        print(preview_df(clinical_df))
        
        # Transform the clinical data to have proper feature names
        clinical_df_transformed = pd.DataFrame()
        clinical_df_transformed[trait] = [0] * len(clinical_df.columns)  # Initialize with default values
        clinical_df_transformed['Age'] = [None] * len(clinical_df.columns)
        clinical_df_transformed['Gender'] = [None] * len(clinical_df.columns)
        
        # Set the indices to be the column names from original clinical_df
        clinical_df_transformed.index = clinical_df.columns
        
        # Based on background information, this dataset doesn't contain chromophobe RCC information
        print("Note: This dataset (GSE40911) focuses on clear cell renal cell carcinoma, not chromophobe RCC.")
        print("The background information states: 'disease: clear cell renal cell carcinoma (RCC)' in row 1")
        is_trait_available = False
    else:
        print("Clinical data file does not exist.")
        is_trait_available = False
        clinical_df_transformed = pd.DataFrame()
except Exception as e:
    print(f"Error loading clinical data: {e}")
    is_trait_available = False
    clinical_df_transformed = pd.DataFrame()

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if not clinical_df_transformed.empty and not normalized_gene_data.empty and is_trait_available:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df_transformed, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have any valid trait values
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 3. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 4. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"The {trait} trait column is not present in the linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical data is missing, genetic data is missing, or trait is unavailable")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
# Based on the dataset context, this dataset doesn't contain chromophobe RCC information
note = "Dataset contains clear cell renal cell carcinoma samples, not chromophobe RCC as required by the trait."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 2488 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE40911.csv

Loading clinical data...
Loaded clinical data with shape: (3, 15)
Clinical data preview:
{'0': [nan, 10.0, nan], '1': [nan, nan, nan], '2': [nan, nan, nan], '3': [nan, nan, nan], '4': [nan, 34.0, nan], '5': [nan, nan, nan], '6': [nan, 6.5, nan], '7': [nan, nan, nan], '8': [nan, nan, nan], '9': [nan, nan, nan], '10': [nan, 2.0, nan], '11': [nan, nan, nan], '12': [nan, nan, nan], '13': [nan, nan, nan], '14': [nan, nan, nan]}
Note: This dataset (GSE40911) focuses on clear cell renal cell carcinoma, not chromophobe RCC.
The background information states: 'disease: clear cell renal cell carcinoma (RCC)' in row 1

Linking clinical and genetic data...
Cannot link data: clinical data is missing, genetic data is missing, or trait is unavailable

Performing final validation...
Abnormality detected in the cohort: GSE40911. Preprocessing failed.
Dataset not usable for Kidney_Chromophobe asso